In [3]:
# loading needed methods
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm

from scipy.fft import fft, dct
from random import seed,sample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_curve, auc,\
precision_score
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler



In [4]:
dataset = pd.read_csv('../input/Fraud/backup.csv') # unchanged dataset1

In [5]:
dataset = dataset.drop('Unnamed: 0', axis = 1)
dataset = pd.get_dummies(dataset,prefix=['type'])

In [6]:
len(dataset[dataset['isFraud']==1])

8213

In [7]:
X = dataset.drop("isFraud",1)
y = dataset.isFraud

In [8]:
yf = dct(X, type=2, norm='ortho')

In [9]:
yf=pd.DataFrame(np.abs(yf))
yf['isFraud']=y

In [10]:
yf.to_csv('Xdct.csv', encoding='utf-8', index = 'false')

In [11]:
def preprocess_data(df):
    class0_df = df[df['isFraud'] == 0] ## majority class, will be used for autoencoder training
    class1_df = df[df['isFraud'] == 1]
    ##
    class0_arr = np.array(class0_df.drop('isFraud',axis = 1)) 
    class1_arr = np.array(class1_df.drop('isFraud',axis = 1))
    X = class0_arr
    X_train, X_test = train_test_split(X, test_size=0.1)
    X_train, X_val = train_test_split(X_train, test_size=0.1)
    
    print("====><>>>>>")
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_val = scaler.transform(X_val)
    return X_train, X_val, X_test, class1_arr

In [12]:
X_train, X_val, X_test, class1_arr = preprocess_data(yf)
n_features = X_train.shape[1]

====><>>>>>


In [13]:
n_features

11

In [14]:
print("Train Size: {}".format(len(X_train)))
print("Val Size: {}".format(len(X_val)))
print("Test Size: {}".format(len(X_test)))
print("No of Features: {}".format(n_features))

Train Size: 2237378
Val Size: 248598
Test Size: 276220
No of Features: 11


In [15]:
len(class1_arr)

8213

In [16]:
# model archticeture

def build_model():
    model = keras.Sequential([
        keras.Input(shape=(n_features,)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Dense(32, activation='relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Dense(16, activation='relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Dense(32, activation='relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Dense(64, activation='relu'),
        keras.layers.BatchNormalization(),
        
        keras.layers.Dense(n_features)
    ])
    
    return model

In [17]:
# model building
model = build_model()

In [18]:
# model compile
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='mse', metrics=['accuracy'])

In [19]:
# callbacks defined

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 5
    lrate = initial_lrate * (drop**((1 + epoch)/epochs_drop))
    return lrate

lrate_scheduler = LearningRateScheduler(step_decay)
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
model_chkpoint = ModelCheckpoint('best_model_dct.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# model fitting
model.fit(X_train, X_train, batch_size=512, epochs=10, validation_data=(X_val, X_val), callbacks=[early_stop, model_chkpoint, lrate_scheduler])

Train on 2237378 samples, validate on 248598 samples
Epoch 1/10
2235392/2237378 [============================>.] - ETA: 0s - loss: 0.0859 - accuracy: 0.4331
Epoch 00001: val_loss improved from inf to 0.03641, saving model to best_model_dct.h5
2237378/2237378 [==============================] - 81s 36us/sample - loss: 0.0859 - accuracy: 0.4332 - val_loss: 0.0364 - val_accuracy: 0.2496
Epoch 2/10
2236416/2237378 [============================>.] - ETA: 0s - loss: 0.0265 - accuracy: 0.5326
Epoch 00002: val_loss improved from 0.03641 to 0.01795, saving model to best_model_dct.h5
2237378/2237378 [==============================] - 71s 32us/sample - loss: 0.0265 - accuracy: 0.5326 - val_loss: 0.0179 - val_accuracy: 0.3453
Epoch 3/10
2236928/2237378 [============================>.] - ETA: 0s - loss: 0.0181 - accuracy: 0.5657
Epoch 00003: val_loss did not improve from 0.01795
2237378/2237378 [==============================] - 84s 38us/sample - loss: 0.0181 - accuracy: 0.5657 - val_loss: 0.0214 - 

In [20]:
def rel(y_test,y_pred):
    predictionsRFR=np.array(y_pred,dtype='int') 
    CM_RFR = confusion_matrix(y_test,predictionsRFR)
    CR_RFR = classification_report(y_test,predictionsRFR)
    fprRFR, recallRFR, thresholdsRFR = roc_curve(y_test, predictionsRFR)
    AUC_RFR = auc(fprRFR, recallRFR)
    print("=============<>==================\n")
    resultsRFR = {"Confusion Matrix":CM_RFR,"Classification Report":CR_RFR,"Area Under Curve":AUC_RFR}
    for measure in resultsRFR:
        print(measure,": \n",resultsRFR[measure])
    print("=============<>==================\n")

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.metrics import precision_recall_curve, roc_curve


def reconstruction_error(actual, pred):
    return np.mean((actual - pred)**2, axis=1)

def evaluate(model, X, y):
    X = X.reshape(-1, n_features)
    out = reconstruction_error(X, model.predict(X))
    print("AUC score: {}".format(roc_auc_score(y, out)))
    print("PR score: {}".format(average_precision_score(y, out)))
    print("\n\n")
    for th in [0.5, 0.6, 0.7, 0.8, 0.9, 1., 1.1 , 1.2, 1.3, 1.4, 1.5, 1.75, 2, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4]:
        out_th = [1 if (o > th) else 0 for o in out]
        print("TH - {}".format(th))
        print("Precision: {}".format(precision_score(y, out_th)))
        print("Recall: {}".format(recall_score(y, out_th)))
        print("F1score: {}".format(f1_score(y, out_th)))
        rel(y,out_th)
        print("\n\n")

In [22]:
final_X_test = np.concatenate((X_test, class1_arr))
final_Y_test = np.concatenate(([0]*len(X_test), [1]*len(class1_arr)))

In [23]:
# Metrics
evaluate(model, final_X_test, final_Y_test)

AUC score: 1.0
PR score: 1.0



TH - 0.5
Precision: 0.942073870153705
Recall: 1.0
F1score: 0.9701730553422716
=============<>==================

Confusion Matrix : 
 [[275715    505]
 [     0   8213]]
Classification Report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    276220
           1       0.94      1.00      0.97      8213

    accuracy                           1.00    284433
   macro avg       0.97      1.00      0.98    284433
weighted avg       1.00      1.00      1.00    284433

Area Under Curve : 
 0.9990858735790312
=============<>==================




TH - 0.6
Precision: 0.9554443927408097
Recall: 1.0
F1score: 0.9772145874234042
=============<>==================

Confusion Matrix : 
 [[275837    383]
 [     0   8213]]
Classification Report : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    276220
           1       0.96      1.00      0.98      8213

    accu